<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart with RAGStack

This notebook demonstrates how to set up a simple RAG pipeline with RAGStack. At the end of this notebook, you will have a fully functioning Question/Answer model that can answer questions using your supplied documents.

A RAG pipeline requires, at minimum, a vector store, an embedding model, and an LLM. In this tutorial, you will use an Astra DB vector store, an OpenAI embedding model, an OpenAI LLM, and LangChain to orchestrate it all together.

## Prerequisites

You will need a vector-enabled Astra database and an OpenAI Account.

* Create an [Astra vector database](https://docs.datastax.com/en/astra-serverless/docs/getting-started/create-db-choices.html).
* Create an [OpenAI account](https://openai.com/)
* Within your database, create an [Astra DB Access Token](https://docs.datastax.com/en/astra-serverless/docs/manage/org/manage-tokens.html) with Database Administrator permissions.
* Get your Astra DB Endpoint:
  * `https://<ASTRA_DB_ID>-<ASTRA_DB_REGION>.apps.astra.datastax.com`

See the [Prerequisites](https://docs.datastax.com/en/ragstack/docs/prerequisites.html) page for more details.

## Setup
`ragstack-ai` includes all the packages you need to build a RAG pipeline.

`datasets` is used to import a sample dataset

In [1]:
# ! pip install -q ragstack-ai datasets

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] = "https://8540165f-d918-4e01-8140-fbc22be05ca8-us-east-2.apps.astra.datastax.com"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## Create RAG Pipeline

### Embedding Model and Vector Store

In [3]:
from langchain_astradb import AstraDBVectorStore
from langchain.embeddings import OllamaEmbeddings
import os

# Configure your embedding model and vector store
embedding = OllamaEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="philocollection",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [5]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[55])

Using the latest cached version of the dataset since datastax/philosopher-quotes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\sunne\.cache\huggingface\datasets\datastax___philosopher-quotes\default\0.0.0\3b22a6a8b08fa444426e3c10a8eb5dfb021b472b (last modified on Fri Jun 28 20:15:21 2024).


An example entry:
{'author': 'schopenhauer', 'quote': 'Every man takes the limits of his own field of vision for the limits of the world.', 'tags': 'ethics;knowledge'}


In [6]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [7]:
docs[2]

Document(page_content='Before you heal the body you must first heal the mind', metadata={'author': 'aristotle', 'ethics': 'y'})

In [8]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs[:20])
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 20 documents.


In [9]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("philocollection").find())

{'data': {'documents': [{'_id': 'e372b04d3b8b46cf9fa591cc4fe075c1', 'content': 'The society that loses its grip on the past is in danger, for it produces men who know nothing but the present, and who are not aware that life had been, and could be, different from what it is.', 'metadata': {'author': 'aristotle', 'history': 'y', 'ethics': 'y', 'knowledge': 'y'}}, {'_id': 'e2f57535957b41fcba29718c43e92a60', 'content': 'You are what you do repeatedly.', 'metadata': {'author': 'aristotle'}}, {'_id': 'ce8dadc201bd471a8079b84aeaf5b42d', 'content': 'The quality of life is determined by its activities.', 'metadata': {'author': 'aristotle'}}, {'_id': '9b71e562855f452788a01eae2c9b1652', 'content': 'The man who is truly good and wise will bear with dignity whatever fortune sends, and will always make the best of his circumstances.', 'metadata': {'author': 'aristotle', 'knowledge': 'y', 'ethics': 'y'}}, {'_id': '1865248ab2eb4341a7aec1e57239475e', 'content': 'Anyone who has no need of anybody but hi

### Basic Retrieval

Retrieve context from your vector database, and pass it to the model with a prompt.

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.llms import Ollama
#load ollama llama3 llm model

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model=Ollama(model='llama3:latest')

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what subject are philosophers most concerned with?")

"Based on the provided context, it seems that philosophers in this case (specifically Aristotle) are most concerned with ethics and knowledge. Both of these documents mention ethics and knowledge as relevant metadata, suggesting that these topics are important to Aristotle's philosophical work."

In [11]:
chain.invoke("from where true happiness comes?")

'According to the provided context, Aristotle says that true happiness comes from "gaining insight and growing into your best possible self." This is stated in two separate documents with identical page content.'

## Cleanup

In [12]:
# WARNING: This will delete the collection and all documents in the collection
vstore.delete_collection()

You now have a fully functioning RAG pipeline! Note that there are several different ways to accomplish this, depending on your input data format, vector store, embedding, model, output type, and more. There are also more advanced RAG techniques that leverage new ingestion, retrieval, and generation patterns.  

RAG is a powerful solution used in tandem with the capabilities of LLMs. Check out our other examples for ideas on how you can build innovative solutions using RAGStack!